# Any and all pacakges used for cleaning and modeling

In [ ]:
import pandas as pd
# !pip install wrds
import wrds
import os
import zipfile

## Part 1: Merging SPAC and IPO data
- Key Points: Self cleaned excel files with offering dates and firm names
- Both read in seperately and merged on the date variabe

In [ ]:
spac_data = pd.read_csv("inputs/SPACs2016-2021.csv", header=0)
IPO_data = pd.read_csv("inputs/IPO_data.csv", header=0)


In [ ]:
IPO_data['CUSIP'] = IPO_data['CUSIP'].apply(lambda x: str(x).zfill(9))
IPO_data

In [ ]:
IPO_data = IPO_data[IPO_data['CUSIP'] != '000000nan']
IPO_data

In [ ]:
spac_data['CUSIP'] = spac_data['CUSIP'].apply(lambda x: str(x).zfill(9))
spac_data

In [ ]:
spac_data = spac_data[spac_data['CUSIP'] != '000000nan']
spac_data

In [ ]:
merged_IPOnSPAC = pd.merge(left=IPO_data, right=spac_data, how='left', on='CUSIP')
merged_IPOnSPAC['SPAC IPO '] = merged_IPOnSPAC['SPAC IPO '].notna().astype(int)
merged_IPOnSPAC = merged_IPOnSPAC.drop(merged_IPOnSPAC.columns[-1], axis=1)
merged_IPOnSPAC.to_csv('inputs/MergedIPO&SPAC.csv')



In [ ]:
# # Convert 'offer date' from 'YYYYMMDD' to 'MM/DD/YYYY'
# IPO_data['offer date'] = pd.to_datetime(IPO_data['offer date'],format='%Y%m%d')
# IPO_data['offer date'] = IPO_data['offer date'].dt.strftime('%m/%d/%Y')

# IPO_data.to_csv("inputs/IPO_data.csv")



In [ ]:
IPO_data.rename(columns={'offer date' : 'Date of IPO'}, inplace=True)

In [ ]:
merged_dates_df = pd.merge(spac_data ,IPO_data, on='Date of IPO', how='inner')

#merged_dates_df.to_csv('inputs/merged_dates.csv')
merged_dates_df

In [ ]:
#Reading in ccm data
ccm_youngfirms = "inputs/ccm_youngfirms_2000_2018.dta/ccm_youngfirms_2000_2018.dta"

if not os.path.exists(ccm_youngfirms):
    zip_path = "inputs/ccm_youngfirms_2000_2018.zip"

    with zipfile.ZipFile(zip_path,'r') as zip_ref:
        zip_ref.extractall("inputs/ccm_youngfirms_2000_2018.dta")
        
ccm_data = data = pd.read_stata(ccm_youngfirms)

In [ ]:
ipo_data = pd.read_csv("inputs/IPO-age(9).csv")

merged_dates_df.rename(columns={'CUSIP' : 'cusip'}, inplace=True)
merge_df_ccm = pd.merge(merged_dates_df, ccm_data, on='cusip', how='inner')

